# size distribution example
Load basic packages

In [1]:
import size_distribution_class as sdc
import statistics

import pandas as pd
import pandas_bokeh
import numpy as np
from bokeh.io import output_file, output_notebook
from bokeh.models import DatetimeTickFormatter, ranges, Span, Label
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.palettes import linear_palette, Turbo256
pd.set_option('plotting.backend', 'pandas_bokeh')
output_notebook(hide_banner=True)
from IPython.core.display import display, HTML


In [2]:
psd = sdc.singeFileProcess( verbose=True )

Please select the directory with the source image tiles.  2.9141 x 2.9141 nm/px
 Analysing C2-C3S 28d cryoBIB_004-cut.tif
--------------------------------------------------
 ...processing particle #500/1114
 ...processing particle #1000/1114
 Analysed particles: 988, ignored 0-values: 126, finished in 929 ms
     file_index   diameter   area  surface        volume
0           0.0   4.296740   14.5     58.0     41.535152
1           0.0   1.128379    1.0      4.0      0.752253
2           0.0   1.595769    2.0      8.0      2.127692
3           0.0   2.256758    4.0     16.0      6.018022
4           0.0   2.393654    4.5     18.0      7.180961
..          ...        ...    ...      ...           ...
983         0.0  29.250931  672.0   2688.0  13104.417162
984         0.0  17.186959  232.0    928.0   2658.249725
985         0.0  31.280738  768.5   3074.0  16026.164935
986         0.0   4.583498   16.5     66.0     50.418476
987         0.0   7.442172   43.5    174.0    215.822980

[988 

basic functions to generate the following diagrams

In [12]:
def get_X_List( values ):
    value_cnt = len(values)
    x_range = list(range(value_cnt))
    x_range.pop(0)
    x_range.append(value_cnt)
    return x_range
    
def getLineDiagram_mean_type2(title, plot_width=1200, legend_location='top_right'):
    p = figure(title=title, y_axis_type="log", plot_width=plot_width, y_range=(1, psd.psd_df['area'].max()), plot_height=600, background_fill_color="#fafafa")
    #p.left[0].formatter.use_scientific = False

    p.xaxis.axis_label = 'area in'
    p.yaxis.axis_label = 'phase content in %'
    
    x_range = get_X_List( list(psd.psd_df['area']) )
    
    colors = linear_palette(Turbo256, 2)

    p.line(x_range, psd.psd_df['area'], line_color=colors[1], line_width=2, legend_label='Test')

    p.legend.location = legend_location
    show(p)

def get_histogram_cld(title, plot_width=1200, legend_location='top_right'):
    p = figure(title=title, y_axis_type="log", plot_width=plot_width, y_range=(1, psd.cld_df['area'].max()), plot_height=600, background_fill_color="#fafafa")
    #p.left[0].formatter.use_scientific = False

    p.xaxis.axis_label = 'area in'
    p.yaxis.axis_label = 'phase content in %'
    
    x_range = get_X_List( list(psd.cld_df['area']) )
    
    colors = linear_palette(Turbo256, 2)

    p.line(x_range, psd.cld_df['area'], line_color=colors[1], line_width=2, legend_label='Test')

    bins = np.linspace(psd.cld_df['area'].min(), psd.cld_df['area'].max(), 1000)
    hist = psd.cld_df.groupby(pd.cut(psd.cld_df['area'], bins))
    print(list(hist[area]))
    #hist = psd.cld_df.hist(column='area',bins=1000)
    #print(hist)
    #p.quad(bottom=0, top=delays['flights'], 
    #    left=delays['left'], right=delays['right'], 
    #    fill_color='red', line_color='black')

    p.legend.location = legend_location
    show(p)

generate Diagrams 

In [4]:
getLineDiagram_mean_type2('repeats per image count')

In [13]:
get_histogram_cld('repeats per image count')

ile_index, diameter, area, surface, volume]
Index: []), (Interval(113519810.909, 113652118.376, closed='right'), Empty DataFrame
Columns: [file_index, diameter, area, surface, volume]
Index: []), (Interval(113652118.376, 113784425.843, closed='right'), Empty DataFrame
Columns: [file_index, diameter, area, surface, volume]
Index: []), (Interval(113784425.843, 113916733.31, closed='right'), Empty DataFrame
Columns: [file_index, diameter, area, surface, volume]
Index: []), (Interval(113916733.31, 114049040.777, closed='right'), Empty DataFrame
Columns: [file_index, diameter, area, surface, volume]
Index: []), (Interval(114049040.777, 114181348.244, closed='right'), Empty DataFrame
Columns: [file_index, diameter, area, surface, volume]
Index: []), (Interval(114181348.244, 114313655.711, closed='right'), Empty DataFrame
Columns: [file_index, diameter, area, surface, volume]
Index: []), (Interval(114313655.711, 114445963.178, closed='right'), Empty DataFrame
Columns: [file_index, diameter, a

In [6]:
len(psd.cld_df)

29637

In [7]:
len(get_X_List( list(psd.cld_df['area']) ))


29637